# Kedro Telemetry with Grafana
> An easy setup that fills the gap of the official documentation

- toc: true 
- badges: true
- comments: true
- author: noklam
- hide: true
- categories: ["python", "kedro"]

## Adding the hooks

https://kedro.readthedocs.io/en/latest/07_extend_kedro/02_hooks.html#add-observability-to-your-pipeline
```python
# <your_project>/src/<your_project>/hooks.py
import sys
from typing import Any, Dict

import statsd
from kedro.framework.hooks import hook_impl
from kedro.pipeline.node import Node


class PipelineMonitoringHooks:
    def __init__(self):
        self._timers = {}
        self._client = statsd.StatsClient(prefix="kedro")

    @hook_impl
    def before_node_run(self, node: Node) -> None:
        node_timer = self._client.timer(node.name)
        node_timer.start()
        self._timers[node.short_name] = node_timer

    @hook_impl
    def after_node_run(self, node: Node, inputs: Dict[str, Any]) -> None:
        self._timers[node.short_name].stop()
        for dataset_name, dataset_value in inputs.items():
            self._client.gauge(dataset_name + "_size", sys.getsizeof(dataset_value))

    @hook_impl
    def after_pipeline_run(self):
        self._client.incr("run")
```

You will need to add this hooks inside your `settings.py`.  
`HOOKS = (ProjectHooks(), PipelineMonitoringHooks())`

## Setup the necessary components
The easiest way to do this is via `Docker`, I have found a Docker Image that just work perfectly locally. For production, you would want to setup the backend properly.

### Grafana (Dashboard)

`docker run -d -p 3000:3000 -p 80:80 grafana/grafana`

### StatsD and Graphite (The database)
```
docker run -d\
 --name graphite\
 --restart=always\
 -p 80:80\
 -p 2003-2004:2003-2004\
 -p 2023-2024:2023-2024\
 -p 8125:8125/udp\
 -p 8126:8126\
 graphiteapp/graphite-statsd


 # https://hub.docker.com/r/graphiteapp/graphite-statsd
```


Then you need to add `Graphite` from `Data Source` via the UI `http://localhost:3000`

Your Graphite URL will either be `http://localhost:80`, `http://127.0.0.1:80` or `http://192.168.0.1:80`. I actually spend hours just to realize my IP was not `localhost`

![Grafana Data Source](images/kedro/grafana.png "Add data source via Grafana UI")